In [1]:
## This notebook ingest the raw "driver_standings" oracle redbull file from the bronze volume to bronze table
### Ingest driver_standings.csv file

In [1]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "delta")
bronze_folder_path    = "/Volumes/f1_bronze/bronze/f1_bronze_volume"
bronze_catalog    = "f1_bronze"
bronze_schema     = "bronze"
bronze_table_dlt = "f1_driver_standings_dlt"
bronze_table_par = "f1_driver_standings_par"

# ----------
##### Step 1 - Read the CSV file using the spark dataframe reader
#  ----------

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

driver_standings_schema = StructType([
    StructField("DRIVERSTANDINGSID", IntegerType(), True),
    StructField("RACEID", IntegerType(), True),
    StructField("DRIVERID", IntegerType(), True),
    StructField("POINTS", FloatType(), True),
    StructField("POSITION", IntegerType(), True),
    StructField("POSITIONTEXT", StringType(), True),
    StructField("WINS", IntegerType(), True)
])


In [1]:
driver_standings_df = spark.read \
.option("header", True) \
.schema(driver_standings_schema) \
.csv(f"{bronze_folder_path}/driver_standings.csv")

In [1]:
display(driver_standings_df)

# ----------
##### Step 2 - Write the output to bronze catalog as parquet file or delta/parquet table
#  ----------

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        driver_standings_df.write.mode("overwrite").parquet(f"{bronze_folder_path}/driver_standings_par")
elif target_type == 'table':
    if target_format == 'parquet':
        driver_standings_df.write.mode("overwrite").format("parquet").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        driver_standings_df.write.mode("overwrite").format("delta").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        driver_standings_read_df = spark.read.parquet(f"{bronze_folder_path}/driver_standings_par")
elif target_type == 'table':
    if target_format == 'parquet':
        driver_standings_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        driver_standings_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")